In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

from llm import get_response

LLM_OVERWRITE = False

/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
meetings_df = pd.read_csv(os.path.join(LOCAL_PATH, "intermediate_data/cpc/meetings-manifest.csv"))
DATES = sorted(list(meetings_df['date']))

In [3]:
PROMPT = """
==== LIST OF AGENDA ITEMS ====

{}

==== DOCUMENT ====

{}

==== PROMPT ====

I just gave you a list of agenda items from a LA City Planning Commission meeting, followed by a document submitted to that meeting. 

Return a response in the following format:


==== YOUR RESPONSE FORMAT ====

TYPE OF DOCUMENT:
<What type of document is it? Your only options are: LETTER OR PETITION, TECHNICAL MODIFICATION OR PROCEDURAL MATTER, SCIENTIFIC OR TECHNICAL REPORT, CV OR BIOGRAPHY, CORRUPTED/ILLEGIBLE/BLANK, TITLE OR SECTION HEADING, OTHER.>

TYPE OF AUTHOR:
<What type of entity wrote the document? Your only options are: INDIVIDUAL, ADVOCACY GROUP, CONSULTANT, LAWYER, DEVELOPER, PUBLIC OFFICIAL, OTHER.>

SUMMARY OF DOCUMENT:
<Summarize the contents of the document.>

REFERENCED AGENDA ITEMS:
<List the agenda items, as a comma delimited list of item numbers, that the submitted document references or is relevant to. If none, say NONE.>

SUPPORT OR OPPOSE:
<Does the submitted document support or oppose the referenced agenda items? Your only options are: DEFINITELY SUPPORT, SOMEWHAT SUPPORT, DEFINITELY OPPOSE, SOMEWHAT OPPOSE, NEUTRAL, NOT RELEVANT.>
"""


In [5]:
# write prompt to figures
with open(os.path.join(LOCAL_PATH, 'figures', 'supplemental_docs_prompt.tex'), 'w') as f:
    out = PROMPT.replace('{}', '<<full agenda text>>', 1)
    out = out.replace('{}', "<<document text>>", 1)
    out = out.strip()
    out = out.replace('\n', '\\\\ \n')
    f.write(out)

In [ ]:
t0 = time.time()
for date in DATES:
    year = date[0:4]
    PATH = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}")
    print(date)

    agenda_file = os.path.join(PATH, 'agenda-items.pkl')
    docs_file = os.path.join(PATH, 'supplemental-docs.pkl')
    output_file = os.path.join(PATH, 'supplemental-docs-summaries.pkl')

    if (not os.path.exists(agenda_file)) or (not os.path.exists(docs_file)):
        continue

    out_df = []

    agenda_df = pd.read_pickle(agenda_file)
    agenda = ""
    for j, jrow in agenda_df.iterrows():
        item_no = jrow['item_no']
        item_title = jrow['title']
        is_casenum = jrow['is_casenum']
        if is_casenum:
            text = jrow['content']
            agenda += f"ITEM NO. {item_no}\n{item_title}\n\n"
            agenda += text
            agenda += "\n\n--------\n\n"
        
    docs_df = pd.read_pickle(docs_file)
    docs_df['skipped'] = docs_df['content'].str.startswith('SKIPPED')
    for j, jrow in docs_df.loc[~docs_df['skipped']].iterrows():
        doc_id = jrow['doc_id']
        start_page = jrow['start_page']
        end_page = jrow['end_page']
        content = jrow['content']
        prompt = PROMPT.format(agenda, content)
        response = get_response(prompt, overwrite=LLM_OVERWRITE)
        msg = response['message']
        perplexity = response['perplexity']
        out_df.append({
            'year': year,
            'date': date,
            'doc_id': doc_id,
            'start_page': start_page,
            'end_page': end_page,
            'prompt': prompt,
            'response': msg,
            'perplexity': perplexity
        })
        print(f"{end_page}... ", end='')

    out_df = pd.DataFrame.from_dict(out_df)
    out_df.to_pickle(output_file)
    print('')
t1 = time.time()
print(f"Elapsed time: {(t1-t0)/60} minutes.")


In [ ]:
i=3
print(out_df.loc[i, 'prompt'])
print('')
print('')
print(out_df.loc[i, 'response'])